# Scientific Tutorials with the Fractional Calculus Library

This notebook provides interactive tutorials for the scientific applications of the `hpfracc` library. We will cover:
1. Anomalous Diffusion
2. EEG Signal Analysis using Fractional Calculus
3. Fractional State Space Modeling


## Tutorial 1: EEG Signal Analysis using Fractional Calculus


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler
import sys
import os

# Adjust the path to import from the root of the project
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..', '..')))

# Import HPFRACC components
from hpfracc.hpfracc.core.derivatives import create_fractional_derivative
from hpfracc.hpfracc.core.integrals import create_fractional_integral
from hpfracc.hpfracc.core.utilities import timing_decorator

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11


class EEGFractionalAnalyzer:
    """
    Comprehensive EEG analyzer using fractional calculus methods.
    """

    def __init__(self, sampling_rate=250):
        """
        Initialize the EEG analyzer.

        Parameters:
        -----------
        sampling_rate : int
            EEG sampling rate in Hz
        """
        self.sampling_rate = sampling_rate
        self.alpha_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.2, 1.5]

        # Initialize fractional operators for different orders
        self.derivatives = {}
        self.integrals = {}

        for alpha in self.alpha_values:
            self.derivatives[alpha] = create_fractional_derivative(
                definition_type="riemann_liouville", alpha=alpha
            )
            self.integrals[alpha] = create_fractional_integral(
                order=alpha, method="RL"
            )

        print(f"EEGFractionalAnalyzer initialized (fs={sampling_rate} Hz)")

    def generate_synthetic_eeg(self, duration=60, noise_level=0.1):
        """
        Generate synthetic EEG data with known fractional properties.

        Parameters:
        -----------
        duration : float
            Duration in seconds
        noise_level : float
            Noise amplitude

        Returns:
        --------
        eeg_data : array
            Synthetic EEG signal
        time : array
            Time vector
        """
        # Time vector
        time = np.arange(0, duration, 1/self.sampling_rate)
        n_samples = len(time)

        # Generate alpha oscillations (8-13 Hz)
        alpha_freq = 10.0
        alpha_osc = np.sin(2 * np.pi * alpha_freq * time)

        # Add long-range dependence using fractional noise
        # This creates 1/f^β noise where β is related to Hurst exponent
        hurst = 0.7  # Long-range dependent
        beta_noise = 2 * hurst - 1

        # Generate fractional noise
        freqs = np.fft.fftfreq(n_samples, 1/self.sampling_rate)
        freqs[0] = 1e-10  # Avoid division by zero

        # Power spectrum: S(f) ∝ 1/f^β
        power_spectrum = 1 / (np.abs(freqs) ** beta_noise)
        power_spectrum[0] = 0  # Zero DC component

        # Generate noise in frequency domain
        phase = np.random.uniform(0, 2*np.pi, n_samples)
        noise_freq = np.sqrt(power_spectrum) * np.exp(1j * phase)
        noise = np.real(np.fft.ifft(noise_freq))

        # Combine components
        eeg_data = alpha_osc + noise_level * noise

        # Add some non-stationarity
        modulation = 1 + 0.3 * \
            np.sin(2 * np.pi * 0.1 * time)  # Slow modulation
        eeg_data *= modulation

        return eeg_data, time

    @timing_decorator
    def fractional_state_space_reconstruction(self, eeg_data, alpha=0.5,
                                              embedding_dim=3, delay=25):
        """
        Perform fractional state space reconstruction using FOSS method.

        Parameters:
        -----------
        eeg_data : array
            EEG signal
        alpha : float
            Fractional order
        embedding_dim : int
            Embedding dimension
        delay : int
            Delay in samples

        Returns:
        --------
        state_space : array
            Reconstructed state space
        """
        n_samples = len(eeg_data)

        # Apply fractional derivative to the signal
        derivative_op = self.derivatives[alpha]

        # Create time vector for derivative computation
        time = np.arange(n_samples) / self.sampling_rate

        # Compute fractional derivative
        def signal_func(t):
            idx = int(t * self.sampling_rate)
            if idx >= n_samples:
                idx = n_samples - 1
            return eeg_data[idx]

        fractional_signal = derivative_op.compute(signal_func, time)

        # Traditional delay embedding with fractional signal
        n_vectors = n_samples - (embedding_dim - 1) * delay
        state_space = np.zeros((n_vectors, embedding_dim))

        for i in range(embedding_dim):
            start_idx = i * delay
            end_idx = start_idx + n_vectors
            state_space[:, i] = fractional_signal[start_idx:end_idx]

        return state_space

    def compute_hurst_exponent(self, eeg_data, method='rs'):
        """
        Compute Hurst exponent using various methods.

        Parameters:
        -----------
        eeg_data : array
            EEG signal
        method : str
            'rs' for R/S analysis, 'dfa' for Detrended Fluctuation Analysis

        Returns:
        --------
        hurst : float
            Hurst exponent
        """
        if method == 'rs':
            return self._rs_analysis(eeg_data)
        elif method == 'dfa':
            return self._dfa_analysis(eeg_data)
        else:
            raise ValueError(f"Unknown method: {method}")

    def _rs_analysis(self, eeg_data):
        """
        R/S (Rescaled Range) analysis for Hurst exponent estimation.
        """
        n = len(eeg_data)
        scales = np.logspace(1, np.log10(n//4), 20, dtype=int)
        rs_values = []

        for scale in scales:
            n_segments = n // scale
            rs_segments = []

            for i in range(n_segments):
                segment = eeg_data[i*scale:(i+1)*scale]

                # Compute mean
                mean_seg = np.mean(segment)

                # Compute cumulative deviation
                dev = segment - mean_seg
                cumdev = np.cumsum(dev)

                # Compute R (range)
                R = np.max(cumdev) - np.min(cumdev)

                # Compute S (standard deviation)
                S = np.std(segment)

                if S > 0:
                    rs_segments.append(R / S)

            if rs_segments:
                rs_values.append(np.mean(rs_segments))

        # Fit power law: R/S ∝ scale^H
        log_scales = np.log(scales[:len(rs_values)])
        log_rs = np.log(rs_values)

        slope, _, _, _, _ = linregress(log_scales, log_rs)
        hurst = slope

        return hurst

    def _dfa_analysis(self, eeg_data):
        """
        Detrended Fluctuation Analysis for Hurst exponent estimation.
        """
        n = len(eeg_data)
        scales = np.logspace(1, np.log10(n//4), 20, dtype=int)
        f_values = []

        for scale in scales:
            n_segments = n // scale
            f_segments = []

            for i in range(n_segments):
                segment = eeg_data[i*scale:(i+1)*scale]

                # Integrate the signal
                integrated = np.cumsum(segment - np.mean(segment))

                # Fit polynomial trend
                x = np.arange(scale)
                coeffs = np.polyfit(x, integrated, 1)
                trend = np.polyval(coeffs, x)

                # Detrend
                detrended = integrated - trend

                # Compute fluctuation
                f_segments.append(np.sqrt(np.mean(detrended**2)))

            if f_segments:
                f_values.append(np.mean(f_segments))

        # Fit power law: F ∝ scale^H
        log_scales = np.log(scales[:len(f_values)])
        log_f = np.log(f_values)

        slope, _, _, _, _ = linregress(log_scales, log_f)
        hurst = slope

        return hurst

    def compute_fractal_dimension(self, eeg_data, method='box'):
        """
        Compute fractal dimension using various methods.

        Parameters:
        -----------
        eeg_data : array
            EEG signal
        method : str
            'box' for box-counting, 'correlation' for correlation dimension

        Returns:
        --------
        fd : float
            Fractal dimension
        """
        if method == 'box':
            return self._box_counting_dimension(eeg_data)
        elif method == 'correlation':
            return self._correlation_dimension(eeg_data)
        else:
            raise ValueError(f"Unknown method: {method}")

    def _box_counting_dimension(self, eeg_data):
        """
        Box-counting dimension estimation.
        """
        # Create phase space using delay embedding
        delay = 25
        embedding_dim = 3
        state_space = self.fractional_state_space_reconstruction(
            eeg_data, alpha=0.5, embedding_dim=embedding_dim, delay=delay
        )

        # Normalize to unit cube
        scaler = StandardScaler()
        state_space_norm = scaler.fit_transform(state_space)

        # Box counting
        scales = np.logspace(-2, 0, 20)
        counts = []

        for scale in scales:
            # Count boxes containing points
            n_boxes = int(1 / scale)
            box_count = 0

            for i in range(n_boxes):
                for j in range(n_boxes):
                    for k in range(n_boxes):
                        # Check if any point falls in this box
                        mask = ((state_space_norm[:, 0] >= i*scale) &
                                (state_space_norm[:, 0] < (i+1)*scale) &
                                (state_space_norm[:, 1] >= j*scale) &
                                (state_space_norm[:, 1] < (j+1)*scale) &
                                (state_space_norm[:, 2] >= k*scale) &
                                (state_space_norm[:, 2] < (k+1)*scale))

                        if np.any(mask):
                            box_count += 1

            counts.append(box_count)

        # Fit power law: N(ε) ∝ ε^(-D)
        log_scales = np.log(scales)
        log_counts = np.log(counts)

        slope, _, _, _, _ = linregress(log_scales, log_counts)
        fd = -slope

        return fd

    def _correlation_dimension(self, eeg_data):
        """
        Correlation dimension estimation.
        """
        # Create phase space
        state_space = self.fractional_state_space_reconstruction(
            eeg_data, alpha=0.5, embedding_dim=3, delay=25
        )

        # Compute correlation integral
        distances = []
        n_points = min(1000, len(state_space))

        for i in range(n_points):
            for j in range(i+1, n_points):
                dist = np.linalg.norm(state_space[i] - state_space[j])
                distances.append(dist)

        distances = np.array(distances)

        # Compute correlation integral for different radii
        radii = np.logspace(-3, 0, 20)
        c_values = []

        for r in radii:
            c = np.sum(distances < r) / len(distances)
            c_values.append(c)

        # Fit power law: C(r) ∝ r^ν
        log_radii = np.log(radii)
        log_c = np.log(c_values)

        # Use only points where C(r) > 0
        valid_idx = log_c > -10
        if np.sum(valid_idx) > 5:
            slope, _, _, _, _ = linregress(
                log_radii[valid_idx], log_c[valid_idx])
            fd = slope
        else:
            fd = 0

        return fd

    def extract_fractional_features(self, eeg_data):
        """
        Extract comprehensive fractional features from EEG signal.

        Parameters:
        -----------
        eeg_data : array
            EEG signal

        Returns:
        --------
        features : dict
            Dictionary of extracted features
        """
        features = {}

        # 1. Hurst exponent (long-range dependence)
        features['hurst_rs'] = self.compute_hurst_exponent(eeg_data, 'rs')
        features['hurst_dfa'] = self.compute_hurst_exponent(eeg_data, 'dfa')

        # 2. Fractal dimension
        features['fractal_dim_box'] = self.compute_fractal_dimension(
            eeg_data, 'box')
        features['fractal_dim_corr'] = self.compute_fractal_dimension(
            eeg_data, 'correlation')

        # 3. Fractional derivatives at different orders
        for alpha in [0.3, 0.5, 0.7]:
            derivative_op = self.derivatives[alpha]
            time = np.arange(len(eeg_data)) / self.sampling_rate

            def signal_func(t):
                idx = int(t * self.sampling_rate)
                if idx >= len(eeg_data):
                    idx = len(eeg_data) - 1
                return eeg_data[idx]

            frac_deriv = derivative_op.compute(signal_func, time)
            features[f'frac_deriv_alpha_{alpha}_mean'] = np.mean(frac_deriv)
            features[f'frac_deriv_alpha_{alpha}_std'] = np.std(frac_deriv)
            features[f'frac_deriv_alpha_{alpha}_max'] = np.max(frac_deriv)

        # 4. Spectral features
        freqs, psd = signal.welch(
            eeg_data, fs=self.sampling_rate, nperseg=1024)

        # Alpha power (8-13 Hz)
        alpha_mask = (freqs >= 8) & (freqs <= 13)
        features['alpha_power'] = np.mean(psd[alpha_mask])

        # Beta power (13-30 Hz)
        beta_mask = (freqs >= 13) & (freqs <= 30)
        features['beta_power'] = np.mean(psd[beta_mask])

        # Theta power (4-8 Hz)
        theta_mask = (freqs >= 4) & (freqs <= 8)
        features['theta_power'] = np.mean(psd[theta_mask])

        # Gamma power (30-100 Hz)
        gamma_mask = (freqs >= 30) & (freqs <= 100)
        features['gamma_power'] = np.mean(psd[gamma_mask])

        # 5. Spectral slope (1/f^β)
        log_freqs = np.log(freqs[freqs > 0])
        log_psd = np.log(psd[freqs > 0])
        slope, _, _, _, _ = linregress(log_freqs, log_psd)
        features['spectral_slope'] = slope

        # 6. Entropy measures
        features['shannon_entropy'] = -np.sum(psd * np.log(psd + 1e-10))
        features['spectral_entropy'] = - \
            np.sum(psd * np.log(psd + 1e-10)) / np.log(len(psd))

        return features

    def classify_cognitive_state(self, eeg_data):
        """
        Classify cognitive state based on fractional features.

        Parameters:
        -----------
        eeg_data : array
            EEG signal

        Returns:
        --------
        state : str
            Classified cognitive state
        confidence : float
            Classification confidence
        """
        # Extract features
        features = self.extract_fractional_features(eeg_data)

        # Simple rule-based classification
        # These thresholds are based on literature but would need calibration
        # for specific applications

        # Alpha dominance suggests relaxed/eyes-closed state
        alpha_ratio = features['alpha_power'] / \
            (features['beta_power'] + 1e-10)

        # High Hurst exponent suggests long-range dependence
        hurst_avg = (features['hurst_rs'] + features['hurst_dfa']) / 2

        # High fractal dimension suggests complex dynamics
        fd_avg = (features['fractal_dim_box'] +
                  features['fractal_dim_corr']) / 2

        # Classification rules
        if alpha_ratio > 2.0 and hurst_avg > 0.6:
            state = "Relaxed/Eyes Closed"
            confidence = min(0.9, alpha_ratio / 3.0)
        elif features['beta_power'] > features['alpha_power'] and fd_avg > 2.0:
            state = "Active/Concentrated"
            confidence = min(
                0.8, features['beta_power'] / features['alpha_power'])
        elif hurst_avg < 0.5:
            state = "Fatigued/Drowsy"
            confidence = 0.7
        else:
            state = "Neutral/Awake"
            confidence = 0.6

        return state, confidence

    def plot_analysis_results(self, eeg_data, time, features, state, confidence):
        """
        Plot comprehensive analysis results.
        """
        fig, axes = plt.subplots(3, 3, figsize=(16, 12))

        # Plot 1: Raw EEG signal
        axes[0, 0].plot(time, eeg_data, 'b-', linewidth=0.8)
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Amplitude')
        axes[0, 0].set_title('Raw EEG Signal')
        axes[0, 0].grid(True)

        # Plot 2: Power spectral density
        freqs, psd = signal.welch(
            eeg_data, fs=self.sampling_rate, nperseg=1024)
        axes[0, 1].semilogy(freqs, psd, 'r-', linewidth=1.5)
        axes[0, 1].set_xlabel('Frequency (Hz)')
        axes[0, 1].set_ylabel('Power Spectral Density')
        axes[0, 1].set_title('Power Spectrum')
        axes[0, 1].grid(True)
        axes[0, 1].set_xlim(0, 50)

        # Plot 3: Fractional derivatives
        time_short = time[:1000]  # Use first 1000 samples for clarity
        eeg_short = eeg_data[:1000]

        for alpha in [0.3, 0.5, 0.7]:
            derivative_op = self.derivatives[alpha]

            def signal_func(t):
                idx = int(t * self.sampling_rate)
                if idx >= len(eeg_short):
                    idx = len(eeg_short) - 1
                return eeg_short[idx]

            frac_deriv = derivative_op.compute(signal_func, time_short)
            axes[0, 2].plot(time_short, frac_deriv, label=f'α={alpha}')

        axes[0, 2].set_xlabel('Time (s)')
        axes[0, 2].set_ylabel('Fractional Derivative')
        axes[0, 2].set_title('Fractional Derivatives')
        axes[0, 2].legend()
        axes[0, 2].grid(True)

        # Plot 4: State space reconstruction
        state_space = self.fractional_state_space_reconstruction(
            eeg_data, alpha=0.5, embedding_dim=3, delay=25
        )
        axes[1, 0].scatter(state_space[:, 0], state_space[:, 1],
                           c=state_space[:, 2], cmap='viridis', alpha=0.6, s=1)
        axes[1, 0].set_xlabel('x(t)')
        axes[1, 0].set_ylabel('x(t+τ)')
        axes[1, 0].set_title('Fractional State Space (2D projection)')
        plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])

        # Plot 5: Hurst exponent analysis
        scales = np.logspace(1, 3, 20, dtype=int)
        rs_values = []

        for scale in scales:
            if scale < len(eeg_data) // 4:
                n_segments = len(eeg_data) // scale
                rs_segments = []

                for i in range(n_segments):
                    segment = eeg_data[i*scale:(i+1)*scale]
                    mean_seg = np.mean(segment)
                    dev = segment - mean_seg
                    cumdev = np.cumsum(dev)
                    R = np.max(cumdev) - np.min(cumdev)
                    S = np.std(segment)
                    if S > 0:
                        rs_segments.append(R / S)

                if rs_segments:
                    rs_values.append(np.mean(rs_segments))
                else:
                    rs_values.append(np.nan)
            else:
                rs_values.append(np.nan)

        valid_idx = ~np.isnan(rs_values)
        if np.sum(valid_idx) > 5:
            axes[1, 1].loglog(scales[valid_idx], np.array(rs_values)[valid_idx], 'bo-')
            axes[1, 1].set_xlabel('Scale')
            axes[1, 1].set_ylabel('R/S')
            axes[1, 1].set_title(
                f'Hurst Analysis (H={features["hurst_rs"]:.3f})')
            axes[1, 1].grid(True)

        # Plot 6: Feature comparison
        feature_names = ['Alpha Power', 'Beta Power',
                         'Theta Power', 'Gamma Power']
        feature_values = [features['alpha_power'], features['beta_power'],
                          features['theta_power'], features['gamma_power']]

        axes[1, 2].bar(feature_names, feature_values, color=[
                       'red', 'blue', 'green', 'orange'])
        axes[1, 2].set_ylabel('Power')
        axes[1, 2].set_title('Spectral Power Distribution')
        axes[1, 2].tick_params(axis='x', rotation=45)

        # Plot 7: Classification results
        axes[2, 0].text(
            0.1, 0.8, f'State: {state}', fontsize=14, fontweight='bold')
        axes[2, 0].text(0.1, 0.6, f'Confidence: {confidence:.2f}', fontsize=12)
        axes[2, 0].text(
            0.1, 0.4, f'Hurst (R/S): {features["hurst_rs"]:.3f}', fontsize=12)
        axes[2, 0].text(
            0.1, 0.2, f'Hurst (DFA): {features["hurst_dfa"]:.3f}', fontsize=12)
        axes[2, 0].set_xlim(0, 1)
        axes[2, 0].set_ylim(0, 1)
        axes[2, 0].set_title('Classification Results')
        axes[2, 0].axis('off')

        # Plot 8: Fractal dimension
        axes[2, 1].scatter(features['fractal_dim_box'], features['fractal_dim_corr'],
                           s=100, c='red', alpha=0.7)
        axes[2, 1].set_xlabel('Box-Counting Dimension')
        axes[2, 1].set_ylabel('Correlation Dimension')
        axes[2, 1].set_title('Fractal Dimensions')
        axes[2, 1].grid(True)

        # Plot 9: Spectral slope
        freqs, psd = signal.welch(
            eeg_data, fs=self.sampling_rate, nperseg=1024)
        log_freqs = np.log(freqs[freqs > 0])
        log_psd = np.log(psd[freqs > 0])

        axes[2, 2].plot(log_freqs, log_psd, 'b-', linewidth=1.5)
        axes[2, 2].set_xlabel('log(Frequency)')
        axes[2, 2].set_ylabel('log(PSD)')
        axes[2, 2].set_title(
            f'Spectral Slope (β={features["spectral_slope"]:.3f})')
        axes[2, 2].grid(True)

        plt.tight_layout()
        plt.show()


def main():
    """
    Main tutorial demonstration.
    """
    print("=" * 60)
    print("TUTORIAL 02: EEG SIGNAL ANALYSIS USING FRACTIONAL CALCULUS")
    print("=" * 60)

    # Initialize analyzer
    analyzer = EEGFractionalAnalyzer(sampling_rate=250)

    # Generate synthetic EEG data
    print("Generating synthetic EEG data...")
    eeg_data, time = analyzer.generate_synthetic_eeg(
        duration=60, noise_level=0.1)

    print(
        f"EEG data generated: {len(eeg_data)} samples, {time[-1]:.1f} seconds")

    # Extract features
    print("Extracting fractional features...")
    features = analyzer.extract_fractional_features(eeg_data)

    print("\nExtracted Features:")
    for key, value in features.items():
        print(f"  {key}: {value:.4f}")

    # Classify cognitive state
    print("\nClassifying cognitive state...")
    state, confidence = analyzer.classify_cognitive_state(eeg_data)

    print(f"Classified State: {state}")
    print(f"Confidence: {confidence:.3f}")

    # Perform state space reconstruction
    print("\nPerforming fractional state space reconstruction...")
    state_space = analyzer.fractional_state_space_reconstruction(
        eeg_data, alpha=0.5, embedding_dim=3, delay=25
    )

    print(f"State space reconstructed: {state_space.shape}")

    # Compute Hurst exponent
    print("\nComputing Hurst exponent...")
    hurst_rs = analyzer.compute_hurst_exponent(eeg_data, 'rs')
    hurst_dfa = analyzer.compute_hurst_exponent(eeg_data, 'dfa')

    print(f"Hurst exponent (R/S): {hurst_rs:.3f}")
    print(f"Hurst exponent (DFA): {hurst_dfa:.3f}")

    # Compute fractal dimension
    print("\nComputing fractal dimension...")
    fd_box = analyzer.compute_fractal_dimension(eeg_data, 'box')
    fd_corr = analyzer.compute_fractal_dimension(eeg_data, 'correlation')

    print(f"Fractal dimension (Box): {fd_box:.3f}")
    print(f"Fractal dimension (Correlation): {fd_corr:.3f}")

    # Plot results
    print("\nGenerating analysis plots...")
    analyzer.plot_analysis_results(eeg_data, time, features, state, confidence)

    # Demonstrate different alpha values
    print("\n--- Analysis with Different Fractional Orders ---")
    for alpha in [0.3, 0.5, 0.7, 1.0]:
        print(f"\nα = {alpha}:")
        state_space_alpha = analyzer.fractional_state_space_reconstruction(
            eeg_data, alpha=alpha, embedding_dim=3, delay=25
        )

        # Compute some statistics
        mean_val = np.mean(state_space_alpha)
        std_val = np.std(state_space_alpha)
        print(f"  Mean: {mean_val:.4f}")
        print(f"  Std:  {std_val:.4f}")

    print(f"\n" + "=" * 60)
    print("TUTORIAL COMPLETE")
    print("=" * 60)


if __name__ == "__main__":
    main()



In [ ]:
if __name__ == "__main__":
    main()


## Tutorial 2: Fractional State Space Modeling


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import sys
import os

# Adjust the path to import from the root of the project
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..', '..')))

# Import HPFRACC components
from hpfracc.hpfracc.core.definitions import FractionalOrder
from hpfracc.hpfracc.core.derivatives import create_fractional_derivative
from hpfracc.hpfracc.core.integrals import create_fractional_integral
from hpfracc.hpfracc.special import gamma
from hpfracc.hpfracc.core.utilities import validate_fractional_order, timing_decorator

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams['font.size'] = 11


class FractionalStateSpaceModel:
    """
    Advanced fractional state space modeling using HPFRACC.
    """

    def __init__(self, alpha=0.5, dim=3):
        """
        Initialize fractional state space model.

        Parameters:
        -----------
        alpha : float
            Fractional order
        dim : int
            State space dimension
        """
        self.alpha = FractionalOrder(alpha)
        self.dim = dim

        # Validate parameters
        if not validate_fractional_order(alpha):
            raise ValueError(f"Invalid fractional order: {alpha}")

        # Initialize fractional operators
        self.derivative = create_fractional_derivative(
            definition_type="riemann_liouville", alpha=alpha
        )
        self.integral = create_fractional_integral(order=alpha, method="RL")

        # Initialize state space matrices
        self.A = None  # State matrix
        self.B = None  # Input matrix
        self.C = None  # Output matrix
        self.D = None  # Feedthrough matrix

        print(f"FractionalStateSpaceModel initialized (α={alpha}, dim={dim})")

    def generate_lorenz_data(self, duration=100, dt=0.01, noise_level=0.01):
        """
        Generate Lorenz system data with fractional dynamics.

        Parameters:
        -----------
        duration : float
            Simulation duration
        dt : float
            Time step
        noise_level : float
            Noise amplitude

        Returns:
        --------
        t : array
            Time vector
        x : array
            State variables
        """
        n_steps = int(duration / dt)
        t = np.linspace(0, duration, n_steps)

        # Lorenz parameters
        sigma = 10.0
        rho = 28.0
        beta_lorenz = 8/3

        # Initialize state
        x = np.zeros((n_steps, 3))
        x[0] = [1.0, 1.0, 1.0]

        # Generate fractional Lorenz dynamics
        for i in range(1, n_steps):
            # Standard Lorenz equations
            dx1 = sigma * (x[i-1, 1] - x[i-1, 0])
            dx2 = x[i-1, 0] * (rho - x[i-1, 2]) - x[i-1, 1]
            dx3 = x[i-1, 0] * x[i-1, 1] - beta_lorenz * x[i-1, 2]

            # Add fractional dynamics
            if self.alpha.alpha != 1.0:
                # Apply fractional derivative effect
                frac_factor = (dt ** (self.alpha.alpha - 1)) / \
                    gamma(self.alpha.alpha)
                dx1 *= frac_factor
                dx2 *= frac_factor
                dx3 *= frac_factor

            # Euler integration
            x[i, 0] = x[i-1, 0] + dt * dx1
            x[i, 1] = x[i-1, 1] + dt * dx2
            x[i, 2] = x[i-1, 2] + dt * dx3

            # Add noise
            x[i] += noise_level * np.random.randn(3)

        # Check for and handle numerical instability
        if not np.all(np.isfinite(x)):
            print("Warning: Numerical instability detected in Lorenz attractor. Replacing non-finite values.")
            x = np.nan_to_num(x)

        return t, x

    @timing_decorator
    def foss_reconstruction(self, time_series, embedding_dim=3, delay=1,
                            alpha_values=None):
        """
        Fractional-Order State Space (FOSS) reconstruction.

        Parameters:
        -----------
        time_series : array
            Input time series
        embedding_dim : int
            Embedding dimension
        delay : int
            Time delay
        alpha_values : list
            Fractional orders to test

        Returns:
        --------
        state_spaces : dict
            Dictionary of reconstructed state spaces for different α
        """
        if alpha_values is None:
            alpha_values = [0.3, 0.5, 0.7, 1.0, 1.3, 1.5]

        state_spaces = {}
        n_samples = len(time_series)

        for alpha in alpha_values:
            # Create fractional derivative operator for this alpha
            derivative_op = create_fractional_derivative(
                definition_type="riemann_liouville", alpha=alpha
            )

            # Apply fractional derivative to time series
            time = np.arange(n_samples)

            def series_func(t):
                idx = int(t)
                if idx >= n_samples:
                    idx = n_samples - 1
                return time_series[idx]

            fractional_series = derivative_op.compute(series_func, time)

            # Traditional delay embedding with fractional signal
            n_vectors = n_samples - (embedding_dim - 1) * delay
            state_space = np.zeros((n_vectors, embedding_dim))

            for i in range(embedding_dim):
                start_idx = i * delay
                end_idx = start_idx + n_vectors
                state_space[:, i] = fractional_series[start_idx:end_idx]

            state_spaces[alpha] = state_space

        return state_spaces

    def mtecm_foss_analysis(self, state_spaces, n_clusters=5):
        """
        Multi-span Transition Entropy Component Method (MTECM-FOSS).

        Parameters:
        -----------
        state_spaces : dict
            Dictionary of state spaces for different α
        n_clusters : int
            Number of clusters for state classification

        Returns:
        --------
        results : dict
            MTECM-FOSS analysis results
        """
        results = {}

        for alpha, state_space in state_spaces.items():
            # Normalize state space
            scaler = StandardScaler()
            state_space_norm = scaler.fit_transform(state_space)

            # Cluster states
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            cluster_labels = kmeans.fit_predict(state_space_norm)

            # Compute transition matrix
            transition_matrix = np.zeros((n_clusters, n_clusters))
            for i in range(len(cluster_labels) - 1):
                current_cluster = cluster_labels[i]
                next_cluster = cluster_labels[i + 1]
                transition_matrix[current_cluster, next_cluster] += 1

            # Normalize transition matrix
            row_sums = transition_matrix.sum(axis=1)
            transition_matrix = transition_matrix / row_sums[:, np.newaxis]
            transition_matrix = np.nan_to_num(transition_matrix, 0)

            # Compute entropy measures
            # Intra-sample entropy (within clusters)
            intra_entropy = 0
            for i in range(n_clusters):
                cluster_points = state_space_norm[cluster_labels == i]
                if len(cluster_points) > 0:
                    # Compute variance within cluster
                    cluster_var = np.var(cluster_points, axis=0)
                    cluster_entropy = np.sum(cluster_var)
                    intra_entropy += cluster_entropy * \
                        len(cluster_points) / len(state_space_norm)

            # Inter-sample entropy (between clusters)
            inter_entropy = 0
            cluster_centers = kmeans.cluster_centers_
            for i in range(n_clusters):
                for j in range(i + 1, n_clusters):
                    distance = np.linalg.norm(
                        cluster_centers[i] - cluster_centers[j])
                    inter_entropy += distance

            # Transition entropy
            transition_entropy = 0
            for i in range(n_clusters):
                for j in range(n_clusters):
                    if transition_matrix[i, j] > 0:
                        transition_entropy -= transition_matrix[i, j] * np.log(
                            transition_matrix[i, j])

            results[alpha] = {
                'intra_entropy': intra_entropy,
                'inter_entropy': inter_entropy,
                'transition_entropy': transition_entropy,
                'total_entropy': intra_entropy + inter_entropy + transition_entropy,
                'transition_matrix': transition_matrix,
                'cluster_labels': cluster_labels,
                'cluster_centers': cluster_centers
            }

        return results

    def estimate_fractional_parameters(self, time_series, method='ls'):
        """
        Estimate parameters of fractional state space model.

        Parameters:
        -----------
        time_series : array
            Input time series
        method : str
            Estimation method ('ls' for least squares, 'kalman' for Kalman filter)

        Returns:
        --------
        params : dict
            Estimated parameters
        """
        if method == 'ls':
            return self._least_squares_estimation(time_series)
        elif method == 'kalman':
            return self._kalman_filter_estimation(time_series)
        else:
            raise ValueError(f"Unknown method: {method}")

    def _least_squares_estimation(self, time_series):
        """
        Least squares parameter estimation.
        """
        n_samples = len(time_series)

        # Create feature matrix
        X = np.zeros((n_samples - 1, 3))
        X[:, 0] = time_series[:-1]  # Previous value
        X[:, 1] = np.arange(n_samples - 1)  # Time
        X[:, 2] = 1  # Constant term

        # Target vector (fractional derivative)
        time = np.arange(n_samples)

        def series_func(t):
            idx = int(t)
            if idx >= n_samples:
                idx = n_samples - 1
            return time_series[idx]

        frac_deriv = self.derivative(series_func, time[:-1])
        y = frac_deriv

        # Solve least squares problem
        params, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)

        return {
            'A': params[0],
            'B': params[1],
            'C': params[2],
            'residuals': residuals[0] if len(residuals) > 0 else 0
        }

    def _kalman_filter_estimation(self, time_series):
        """
        Kalman filter parameter estimation (simplified).
        """
        # Simplified Kalman filter implementation
        n_samples = len(time_series)

        # Initialize state and covariance
        x_est = np.zeros(self.dim)
        P = np.eye(self.dim) * 0.1

        # Process and measurement noise
        Q = np.eye(self.dim) * 0.01
        R = 0.1

        # Storage
        state_history = np.zeros((n_samples, self.dim))

        for k in range(n_samples):
            # Prediction step
            # Simplified state transition (would be more complex in practice)
            x_pred = x_est
            P_pred = P + Q

            # Update step
            # Simplified measurement model
            H = np.array([1, 0, 0])  # Only first state is observed
            y = time_series[k]

            # Kalman gain
            K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)

            # State update
            x_est = x_pred + K * (y - H @ x_pred)
            P = (np.eye(self.dim) - K @ H) @ P_pred

            state_history[k] = x_est

        return {
            'state_history': state_history,
            'final_state': x_est,
            'final_covariance': P
        }

    def stability_analysis(self, A_matrix=None):
        """
        Analyze stability of fractional state space system.

        Parameters:
        -----------
        A_matrix : array, optional
            State matrix (if None, uses estimated matrix)

        Returns:
        --------
        stability_info : dict
            Stability analysis results
        """
        if A_matrix is None:
            # Use identity matrix as default
            A_matrix = np.eye(self.dim)

        # Compute eigenvalues
        eigenvals = linalg.eigvals(A_matrix)

        # For fractional systems, stability condition is more complex
        # |arg(λ)| > απ/2 for all eigenvalues λ

        alpha_threshold = self.alpha.alpha * np.pi / 2
        stability_margins = []

        for eig in eigenvals:
            arg_eig = np.abs(np.angle(eig))
            margin = arg_eig - alpha_threshold
            stability_margins.append(margin)

        # Determine stability
        min_margin = min(stability_margins)
        is_stable = min_margin > 0

        # Compute stability measures
        stability_radius = min(np.abs(eigenvals))
        condition_number = linalg.cond(A_matrix)

        return {
            'eigenvalues': eigenvals,
            'stability_margins': stability_margins,
            'is_stable': is_stable,
            'min_stability_margin': min_margin,
            'stability_radius': stability_radius,
            'condition_number': condition_number,
            'alpha_threshold': alpha_threshold
        }

    def simulate_fractional_system(self, u, x0=None, dt=0.01):
        """
        Simulate fractional state space system.

        Parameters:
        -----------
        u : array
            Input signal
        x0 : array, optional
            Initial state
        dt : float
            Time step

        Returns:
        --------
        t : array
            Time vector
        x : array
            State history
        y : array
            Output history
        """
        n_steps = len(u)
        t = np.arange(n_steps) * dt

        # Initialize state
        if x0 is None:
            x0 = np.zeros(self.dim)

        x = np.zeros((n_steps, self.dim))
        y = np.zeros(n_steps)
        x[0] = x0

        # Use default matrices if not set
        if self.A is None:
            self.A = -np.eye(self.dim) * 0.1
        if self.B is None:
            self.B = np.ones(self.dim)
        if self.C is None:
            self.C = np.array([1, 0, 0])
        if self.D is None:
            self.D = 0

        # Simulation loop
        for k in range(1, n_steps):
            # Fractional state equation
            # D^α x(t) = Ax(t) + Bu(t)

            # Simplified fractional integration
            # In practice, this would use proper fractional integration
            frac_factor = (dt ** self.alpha.alpha) / \
                gamma(self.alpha.alpha + 1)

            # State update
            dx = self.A @ x[k-1] + self.B * u[k-1]
            x[k] = x[k-1] + frac_factor * dx

            # Output equation
            y[k] = self.C @ x[k] + self.D * u[k]

        return t, x, y

    def plot_foss_analysis(self, state_spaces, mtecm_results):
        """
        Plot comprehensive FOSS analysis results.
        """
        fig, axes = plt.subplots(3, 3, figsize=(18, 15))

        # Plot 1: State spaces for different α
        alphas = list(state_spaces.keys())
        colors = plt.cm.viridis(np.linspace(0, 1, len(alphas)))

        for i, alpha in enumerate(alphas):
            state_space = state_spaces[alpha]
            axes[0, 0].scatter(state_space[:, 0], state_space[:, 1],
                               c=colors[i], alpha=0.6, s=1, label=f'α={alpha}')

        axes[0, 0].set_xlabel('x(t)')
        axes[0, 0].set_ylabel('x(t+τ)')
        axes[0, 0].set_title('FOSS Reconstruction')
        axes[0, 0].legend()
        axes[0, 0].grid(True)

        # Plot 2: Entropy measures vs α
        alphas = list(mtecm_results.keys())
        intra_entropy = [mtecm_results[alpha]['intra_entropy']
                         for alpha in alphas]
        inter_entropy = [mtecm_results[alpha]['inter_entropy']
                         for alpha in alphas]
        transition_entropy = [mtecm_results[alpha]
                              ['transition_entropy'] for alpha in alphas]
        total_entropy = [mtecm_results[alpha]['total_entropy']
                         for alpha in alphas]

        axes[0, 1].plot(alphas, intra_entropy, 'b-o', label='Intra-sample')
        axes[0, 1].plot(alphas, inter_entropy, 'r-s', label='Inter-sample')
        axes[0, 1].plot(alphas, transition_entropy, 'g-^', label='Transition')
        axes[0, 1].plot(alphas, total_entropy, 'k-*', label='Total')
        axes[0, 1].set_xlabel('Fractional Order α')
        axes[0, 1].set_ylabel('Entropy')
        axes[0, 1].set_title('MTECM-FOSS Entropy Analysis')
        axes[0, 1].legend()
        axes[0, 1].grid(True)

        # Plot 3: Transition matrices
        best_alpha = alphas[np.argmax(total_entropy)]
        transition_matrix = mtecm_results[best_alpha]['transition_matrix']

        im = axes[0, 2].imshow(
            transition_matrix, cmap='viridis', aspect='auto')
        axes[0, 2].set_xlabel('Next State')
        axes[0, 2].set_ylabel('Current State')
        axes[0, 2].set_title(f'Transition Matrix (α={best_alpha})')
        plt.colorbar(im, ax=axes[0, 2])

        # Plot 4: State clustering
        cluster_labels = mtecm_results[best_alpha]['cluster_labels']
        cluster_centers = mtecm_results[best_alpha]['cluster_centers']
        state_space = state_spaces[best_alpha]

        scatter = axes[1, 0].scatter(state_space[:, 0], state_space[:, 1],
                                     c=cluster_labels, cmap='tab10', alpha=0.6, s=1)
        axes[1, 0].scatter(cluster_centers[:, 0], cluster_centers[:, 1],
                           c='red', s=100, marker='x', linewidths=3)
        axes[1, 0].set_xlabel('x(t)')
        axes[1, 0].set_ylabel('x(t+τ)')
        axes[1, 0].set_title(f'State Clustering (α={best_alpha})')
        plt.colorbar(scatter, ax=axes[1, 0])

        # Plot 5: Stability analysis
        stability_info = self.stability_analysis()
        eigenvals = stability_info['eigenvalues']

        # Plot eigenvalues in complex plane
        axes[1, 1].scatter(eigenvals.real, eigenvals.imag, c='red', s=100)
        axes[1, 1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
        axes[1, 1].axvline(x=0, color='k', linestyle='-', alpha=0.3)

        # Draw stability region
        theta = np.linspace(0, 2*np.pi, 100)
        r = 1
        x_circle = r * np.cos(theta)
        y_circle = r * np.sin(theta)
        axes[1, 1].plot(x_circle, y_circle, 'k--',
                        alpha=0.5, label='Unit Circle')

        axes[1, 1].set_xlabel('Real Part')
        axes[1, 1].set_ylabel('Imaginary Part')
        axes[1, 1].set_title('Eigenvalue Distribution')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
        axes[1, 1].set_aspect('equal')

        # Plot 6: Parameter estimation results
        # Generate test data
        t_test = np.linspace(0, 10, 1000)
        u_test = np.sin(2 * np.pi * 0.5 * t_test)

        # Simulate system
        t_sim, x_sim, y_sim = self.simulate_fractional_system(u_test)

        axes[1, 2].plot(t_sim, y_sim, 'b-', linewidth=2, label='Output')
        axes[1, 2].plot(t_sim, u_test, 'r--', linewidth=1, label='Input')
        axes[1, 2].set_xlabel('Time')
        axes[1, 2].set_ylabel('Amplitude')
        axes[1, 2].set_title('Fractional System Simulation')
        axes[1, 2].legend()
        axes[1, 2].grid(True)

        # Plot 7: Lorenz attractor
        t_lorenz, x_lorenz = self.generate_lorenz_data(duration=50, dt=0.01)

        axes[2, 0].plot(x_lorenz[:, 0], x_lorenz[:, 1], 'b-', linewidth=0.5)
        axes[2, 0].set_xlabel('x')
        axes[2, 0].set_ylabel('y')
        axes[2, 0].set_title('Fractional Lorenz Attractor')
        axes[2, 0].grid(True)

        # Plot 8: Time series analysis
        time_series = x_lorenz[:, 0]  # Use x-component
        state_spaces_lorenz = self.foss_reconstruction(time_series)

        # Plot reconstruction for α=0.5
        if 0.5 in state_spaces_lorenz:
            state_space_05 = state_spaces_lorenz[0.5]
            axes[2, 1].scatter(state_space_05[:, 0], state_space_05[:, 1],
                               c=state_space_05[:, 2], cmap='viridis', alpha=0.6, s=1)
            axes[2, 1].set_xlabel('x(t)')
            axes[2, 1].set_ylabel('x(t+τ)')
            axes[2, 1].set_title('Lorenz FOSS (α=0.5)')
            plt.colorbar(axes[2, 1].collections[0], ax=axes[2, 1])

        # Plot 9: Complexity measures
        complexity_measures = []
        for alpha in alphas:
            if alpha in mtecm_results:
                complexity = mtecm_results[alpha]['total_entropy']
                complexity_measures.append(complexity)

        axes[2, 2].plot(alphas[:len(complexity_measures)],
                        complexity_measures, 'bo-', linewidth=2)
        axes[2, 2].set_xlabel('Fractional Order α')
        axes[2, 2].set_ylabel('Complexity Measure')
        axes[2, 2].set_title('System Complexity vs α')
        axes[2, 2].grid(True)

        plt.tight_layout()
        plt.show()


def main():
    """
    Main tutorial demonstration.
    """
    print("=" * 60)
    print("TUTORIAL 03: FRACTIONAL STATE SPACE MODELING")
    print("=" * 60)

    # Initialize model
    model = FractionalStateSpaceModel(alpha=0.5, dim=3)

    # Generate Lorenz data
    print("Generating fractional Lorenz system data...")
    t_lorenz, x_lorenz = model.generate_lorenz_data(duration=100, dt=0.01)

    print(f"Lorenz data generated: {len(t_lorenz)} time steps")

    # FOSS reconstruction
    print("Performing FOSS reconstruction...")
    time_series = x_lorenz[:, 0]  # Use x-component
    state_spaces = model.foss_reconstruction(
        time_series, embedding_dim=3, delay=10)

    print(
        f"FOSS reconstruction completed for {len(state_spaces)} fractional orders")

    # MTECM-FOSS analysis
    print("Performing MTECM-FOSS analysis...")
    mtecm_results = model.mtecm_foss_analysis(state_spaces, n_clusters=5)

    print("\nMTECM-FOSS Results:")
    for alpha, results in mtecm_results.items():
        print(f"  α={alpha}: Total Entropy = {results['total_entropy']:.4f}")

    # Parameter estimation
    print("\nEstimating fractional parameters...")
    params_ls = model.estimate_fractional_parameters(time_series, method='ls')
    params_kf = model.estimate_fractional_parameters(
        time_series, method='kalman')

    print("Least Squares Estimation:")
    for key, value in params_ls.items():
        if key != 'residuals':
            print(f"  {key}: {value:.4f}")

    # Stability analysis
    print("\nPerforming stability analysis...")
    stability_info = model.stability_analysis()

    print(
        f"System Stability: {'Stable' if stability_info['is_stable'] else 'Unstable'}")
    print(
        f"Min Stability Margin: {stability_info['min_stability_margin']:.4f}")
    print(f"Stability Radius: {stability_info['stability_radius']:.4f}")

    # System simulation
    print("\nSimulating fractional system...")
    t_sim = np.linspace(0, 10, 1000)
    u_sim = np.sin(2 * np.pi * 0.5 * t_sim)

    t_out, x_out, y_out = model.simulate_fractional_system(u_sim)

    print(f"Simulation completed: {len(t_out)} time steps")

    # Plot results
    print("\nGenerating analysis plots...")
    model.plot_foss_analysis(state_spaces, mtecm_results)

    # Demonstrate different fractional orders
    print("\n--- Analysis with Different Fractional Orders ---")
    for alpha in [0.3, 0.5, 0.7, 1.0, 1.3]:
        print(f"\nα = {alpha}:")

        # Create model with different alpha
        model_alpha = FractionalStateSpaceModel(alpha=alpha, dim=3)

        # FOSS reconstruction
        state_spaces_alpha = model_alpha.foss_reconstruction(time_series)

        # MTECM analysis
        mtecm_alpha = model_alpha.mtecm_foss_analysis(state_spaces_alpha)

        # Find best alpha based on total entropy
        best_entropy = max([results['total_entropy']
                           for results in mtecm_alpha.values()])
        print(f"  Best Total Entropy: {best_entropy:.4f}")

        # Stability analysis
        stability_alpha = model_alpha.stability_analysis()
        print(
            f"  Stability: {'Stable' if stability_alpha['is_stable'] else 'Unstable'}")

    print(f"\n" + "=" * 60)
    print("TUTORIAL COMPLETE")
    print("=" * 60)


if __name__ == "__main__":
    main()



In [ ]:
if __name__ == "__main__":
    main()
